In [60]:
import random
from collections import namedtuple, deque
from matplotlib import pyplot as plt

### Problem definition

In [61]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

N = 500
generated_problem = problem(N, 42)

## Genetic Algorithm

### Genetic Operators  

In [62]:
Individual = namedtuple("Individual", ["genome", "fitness"])

PROBLEM_SIZE = len(generated_problem)
POPULATION_SIZE = 30
NUM_GENERATIONS = 200
OFFSPRING_SIZE = 80

def tournament(population, tournament_size=5):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)
    # if random.random() < 0.8 else random.choice(population)

def cross_over(g1, g2):
    cut = random.randint(0, PROBLEM_SIZE)
    return g1[:cut] + g2[cut:]

def cross_over2(g1, g2):
    return tuple(random.choice([g1[e], g2[e]]) for e in range(0, PROBLEM_SIZE))

def mutation(g):
    point = random.randint(0, PROBLEM_SIZE-1)
    return g[:point] + (1 - g[point],) + g[point + 1:]
    


### Evaluate fitness and other useful function

In [63]:
def evaluate_weight(genome):
        indexes = (i for i, x in enumerate(genome) if x == 1)
        all_elements = [element for sublist in [generated_problem[x] for x in indexes] for element in sublist]
        return len(all_elements)

def is_solving(genome):
    indexes = (i for i, x in enumerate(genome) if x == 1)
    all_elements = [element for sublist in (generated_problem[x] for x in indexes) for element in sublist]
    coverage = len(set(all_elements))
    return coverage == N

def evaluate_fitness(genome):
    # fitness considers how many numbers are already covered and the weight of the solution
    indexes = (i for i, x in enumerate(genome) if x == 1)
    all_elements = [element for sublist in (generated_problem[x] for x in indexes) for element in sublist]
    weight = len(all_elements)
    coverage = len(set(all_elements))
    #boost_for_correct_solution = 2*N if coverage == N else 0
    #return boost_for_correct_solution + coverage - 0.5*weight
    return coverage, -weight

#### Plot performance function

In [64]:
def plot_performance(fitness_log):
    off_line = [max(f[1] for f in fitness_log if f[0] == x) / (x + 1) for x in range(NUM_GENERATIONS)]
    on_line = [max(f[1] for f in fitness_log if f[0] <= x) / (x + 1) for x in range(NUM_GENERATIONS)]
    gen_best = [max(f[1] for f in fitness_log if f[0] == x) for x in range(NUM_GENERATIONS)]

    plt.figure(figsize=(15, 6))
    plt.scatter([x for x, _ in fitness_log], [y for _, y in fitness_log], marker=".")
    plt.plot([x for x, _ in enumerate(gen_best)], [y for _, y in enumerate(gen_best)])
    plt.plot([x for x, _ in enumerate(on_line)], [y for _, y in enumerate(on_line)])
    plt.plot([x for x, _ in enumerate(off_line)], [y for _, y in enumerate(off_line)])

### Initial Population generation

In [65]:
def initialize_population():
    population = deque()
    fitness_log = [(0, i.fitness) for i in population]
    ### RANDOM GENERATION
    # for genome in [tuple([random.choice([1, 0]) for _ in range(PROBLEM_SIZE)]) for _ in range(POPULATION_SIZE)]:
    #     population.append(Individual(genome, evaluate_fitness(genome)))

    ### EMPTY GENERATION
    genome0 = tuple(0 for e in range(PROBLEM_SIZE))
    f0 = evaluate_fitness(genome0)
    for genome in range(POPULATION_SIZE):
        population.append(Individual(genome0, f0))

    return population, fitness_log



### Evolution

In [66]:
def run_evolution():
    population, fitness_log = initialize_population()
    count_no_improvement = 0
    for g in range(NUM_GENERATIONS):
        offspring = list()
        lr = 0.2 if g < 200 else 0.7
        for i in range(OFFSPRING_SIZE):
            if random.random() < lr:
                p = tournament(population)
                o = mutation(p.genome)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1.genome, p2.genome)
        offspring.append(Individual(o, evaluate_fitness(o)))
        population += offspring
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]

        print(f"gen: {g}, fitness: {population[0].fitness}, weight: {evaluate_weight(population[0].genome)}, solves? {is_solving(population[0].genome)}")
    return [population[0].fitness, evaluate_weight(population[0].genome)]


In [67]:
res = run_evolution()

print(f"fitness: {res[0]}, weight: {res[1]}")

gen: 0, fitness: (100, -100), weight: 100, solves? False
gen: 1, fitness: (100, -100), weight: 100, solves? False
gen: 2, fitness: (100, -100), weight: 100, solves? False
gen: 3, fitness: (144, -144), weight: 144, solves? False
gen: 4, fitness: (144, -144), weight: 144, solves? False
gen: 5, fitness: (144, -144), weight: 144, solves? False
gen: 6, fitness: (144, -144), weight: 144, solves? False
gen: 7, fitness: (156, -156), weight: 156, solves? False
gen: 8, fitness: (156, -156), weight: 156, solves? False
gen: 9, fitness: (156, -156), weight: 156, solves? False
gen: 10, fitness: (156, -156), weight: 156, solves? False
gen: 11, fitness: (156, -156), weight: 156, solves? False
gen: 12, fitness: (156, -156), weight: 156, solves? False
gen: 13, fitness: (156, -156), weight: 156, solves? False
gen: 14, fitness: (156, -156), weight: 156, solves? False
gen: 15, fitness: (156, -156), weight: 156, solves? False
gen: 16, fitness: (156, -156), weight: 156, solves? False
gen: 17, fitness: (156, 